https://github.com/openai/swarm  

In [1]:
import os
import sys
import warnings; warnings.filterwarnings("ignore")
import json
import requests
import torch as th

from pprint import pp
from dotenv import load_dotenv
from openai import OpenAI
from swarm import (Swarm, Agent)
from IPython.display import (Markdown, display)

In [2]:
if sys.platform == "darwin":
    device = th.device("mps")
else:
    device = th.device("cuda" if th.cuda.is_available() else "cpu")
    
devive_cnt = th.cuda.device_count()
print(f"device = {device}; devive_cnt = {devive_cnt}")
print(f"torch = {th.__version__}")
print(f"cuda = {th.version.cuda}")

device = cuda; devive_cnt = 1
torch = 2.5.1+cu121
cuda = 12.1


In [3]:
path_project = "C:/my_project/MyGit/Machine-Learning-Column/hugging_face"
path_data = os.path.join(os.path.dirname(path_project), "data")
path_output = os.path.join(os.path.dirname(path_project), "output")

if sys.platform == "darwin":
    path_model = "/Users/lukasi33/project/LLM"
else:
    path_model = "F:/LLM"

## step-1: 载入 API KEY

In [4]:
load_dotenv(dotenv_path="explore.env")
deepseek_key = os.getenv("DEEPSEEK_KEY")
baidu_key = os.getenv("BAIDU_KEY")

## step-2: 实例化客户端

In [5]:
# 调用模型 API
# client = OpenAI(
#     api_key=deepseek_key,
#     base_url="https://api.deepseek.com/"
#     )

In [5]:
# 本地 Ollama
client = OpenAI(
    base_url="http://127.0.0.1:11434/v1",
    api_key="EMPTY",
    timeout=60
    )

In [6]:
swarm_client = Swarm(client)

## step-3: Agent 推理

In [7]:
# checkpoint = "deepseek-chat"  # v3
# checkpoint = "deepseek-reasoner"  # r1
# checkpoint = "deepseek-r1:14b"
# checkpoint = "qwen2.5:7b"
checkpoint = "qwen2.5:14b"

In [8]:
agent = Agent(
    name="Agent",
    model=checkpoint,
    instructions="You are a helpful assistant on business travel."
)

In [9]:
system_prompt = (
    "你叫小慧助手，是由Lukas开发的差旅智能客服。"
    "你的身份是一名差旅秘书，"
    "你的任务是为用户提供基础对话、差旅知识问答、酒店推荐服务。"
    "当问及你的模型参数时，标准回答是属于公司保密信息，要强调模型设计的高效，能够提供高质量的服务。"
    "You are a helpful assistant on business travel."
)

In [10]:
user_prompt = "你好呀~"

In [11]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [12]:
response = swarm_client.run(
    agent=agent,
    messages=messages
)

In [13]:
print(response.messages[-1]["content"])

您好！有什么可以帮助您的吗？无论是关于旅行规划、酒店预订还是其他差旅相关的问题，都可以随时问我哦。


## step-4: 工具调用

In [84]:
cityName2districtId = {
    "北京": "110100",
    "上海": "310100",
    "广州": "440100",
    "深圳": "440300",
    "南京": "320100",
    "杭州": "330100"
}

In [15]:
def get_weather(cityName):
    districtId = cityName2districtId.get(cityName)
    url = f"https://api.map.baidu.com/weather/v1/?district_id={districtId}&data_type=all&ak={baidu_key}"
    response = requests.get(url)
    data = response.json()
    return json.dumps(data)

In [16]:
# test tool
data = get_weather(cityName="南京")
data

'{"status": 0, "result": {"location": {"country": "\\u4e2d\\u56fd", "province": "\\u6c5f\\u82cf\\u7701", "city": "\\u5357\\u4eac\\u5e02", "name": "\\u5357\\u4eac", "id": "320100"}, "now": {"text": "\\u591a\\u4e91", "temp": 7, "feels_like": 6, "rh": 80, "wind_class": "2\\u7ea7", "wind_dir": "\\u4e1c\\u5317\\u98ce", "uptime": "20250201190000"}, "forecasts": [{"text_day": "\\u9634", "text_night": "\\u5c0f\\u96e8", "high": 9, "low": 4, "wc_day": "3~4\\u7ea7", "wd_day": "\\u5317\\u98ce", "wc_night": "3~4\\u7ea7", "wd_night": "\\u4e1c\\u5317\\u98ce", "date": "2025-02-01", "week": "\\u661f\\u671f\\u516d"}, {"text_day": "\\u9634", "text_night": "\\u591a\\u4e91", "high": 9, "low": 3, "wc_day": "<3\\u7ea7", "wd_day": "\\u5317\\u98ce", "wc_night": "<3\\u7ea7", "wd_night": "\\u5317\\u98ce", "date": "2025-02-02", "week": "\\u661f\\u671f\\u65e5"}, {"text_day": "\\u591a\\u4e91", "text_night": "\\u6674", "high": 8, "low": -1, "wc_day": "3~4\\u7ea7", "wd_day": "\\u5317\\u98ce", "wc_night": "<3\\u7ea7",

In [17]:
agent = Agent(
    name="Master Agent",
    model=checkpoint,
    instructions="You are a helpful assistant on business travel.",
    functions=[get_weather]
)

In [18]:
# 模拟场景-1
user_prompt = "帮我查下南京的天气"

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [19]:
response = swarm_client.run(
    agent=agent,
    messages=messages
)

In [20]:
print(response.messages[-1]["content"])

南京今天的天气是多云，白天的温度为7℃。晚上预计有小雨夹雪，最低气温为4℃左右。明天天气持续阴天转晴冷，最高温度大约在9度。

未来几天的具体情况如下：
- 2025年2月1日（周五）白天有雨夹雪，夜间小到中雪，最低温4℃；
- 2025年2月2日（周六）与一日类似，天气阴沉，最高温度可能达到9度。
- 预计在2月3日至4日期间会经历一次冷空气带来的降温过程，会有雨夹雪转雪的天气变化。
请根据预报及时调整外出着装和行程安排。


In [28]:
# 模拟场景-2
departCityName = "南京"
arriveCityName = "深圳"
startDate = "2025-02-02"
endDate = "2025-02-10"
travelPurpose = "深圳基地培训"
query = "我明天要去深圳出差，帮我看下天气是否适合出差"

user_prompt = (
    f"出发城市：{departCityName}\n"
    f"到达城市：{arriveCityName}\n"
    f"出差时间：{startDate}至{endDate}\n"
    f"出差目的：{travelPurpose}\n"
    f"需求备注：{query}"
)

In [29]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [30]:
response = swarm_client.run(
    agent=agent,
    messages=messages
)

In [31]:
print(response.messages[-1]["content"])

根据天气预报，您计划出差期间（2月2日至2月10日）深圳的天气大部分时间是多云和小雨。其中：

- 2月2日：白天为多云，晚上转为小雨；气温较低，最高温度约为22℃，最低温度为14℃。
- 2月3日至2月5日：都是以多云为主，并且有一定概率下雨，早晚温差较大，平均在18到11摄氏度之间。

看起来大部分时间天气适合出差。但请注意随身携带雨具，保持体温舒适。如果培训行程较为户外或重要，请适当考虑穿着与保暖措施以及防护用品准备。祝您在深圳愉快，并顺利完成任务！


## step-5: Multi-Agent

In [81]:
def get_weather(cityName):
    """
    cityName: 城市名称
    """
    districtId = cityName2districtId.get(cityName)
    url = f"https://api.map.baidu.com/weather/v1/?district_id={districtId}&data_type=all&ak={baidu_key}"
    response = requests.get(url)
    data = response.json()
    return json.dumps(data)

def get_distance(poiNameA, poiNameB):
    return "查询从A点到B点的距离与路线"

# def transfer_to_get_weather():
#     return get_weather

# def transfer_to_get_distance():
#     return get_distance

In [82]:
dialog_agent = Agent(
    name="基本对话助手（dialog_agent）",
    model=checkpoint,
    instructions=(
        "你的任务是与用户进行基本的对话问答，包括：<闲聊>、<查询天气>、<查询距离或路线>。"
        "当用户的意图是<闲聊>时，不用调用工具，直接以差旅秘书的身份与用户聊天；"
        "当用户的意图是<查询天气>时，调用<get_weather>工具，并为用户提供有价值的建议；"
        "当用户的意图是<查询距离或路线>时，调用<get_distance>工具，并为用户提供有价值的建议。"
        "注意事项：不要与用户分享你的思维过程！不要擅自替用户做出不合理的假设！"
    ),
    functions=[get_weather, get_distance]
)

def transfer_to_dialog_agent():
    return dialog_agent

In [11]:
def get_enterprise_policy(query):
    return "根据用户query查询企业差旅政策与制度"

def get_hotel_info(hotelName):
    return "根据hotelName查询酒店设施与政策等信息"

def get_faq(query):
    return "根据用户query查询差旅业务FAQ信息"

# def transfer_to_get_enterprise_policy():
#     return get_enterprise_policy

# def transfer_to_get_hotel_info():
#     return get_hotel_info

# def transfer_to_get_faq():
#     return get_faq

In [63]:
knowledge_agent = Agent(
    name="差旅知识问答助手（knowledge_agent）",
    model=checkpoint,
    instructions=(
        "你的任务是为用户提供差旅领域的专业知识问答，包括：<企业差旅政策与制度>、<酒店设施与政策>、<差旅业务FAQ>。"
        "当用户的意图是询问<企业差旅政策与制度>时，调用<get_enterprise_policy>工具，并为用户提供有价值的建议；"
        "当用户的意图是询问<酒店设施与政策>时，调用<get_hotel_info>工具，并为用户提供有价值的建议；"
        "当用户的意图是询问<差旅业务FAQ>时，如住宿问题、机票问题、打车问题等，调用<get_faq>工具，并为用户提供有价值的建议。"
        "注意事项：不要与用户分享你的思维过程！不要擅自替用户做出不合理的假设！"
    ),
    functions=[get_enterprise_policy, get_hotel_info, get_faq]
)

def transfer_to_knowledge_agent():
    return knowledge_agent

In [13]:
def get_recommend_hotel(query):
    return "根据用户query推荐酒店"

def get_similar_hotel(query):
    return "根据用户query找相似酒店"

# def transfer_to_get_recommend_hotel():
#     return get_recommend_hotel

# def transfer_to_get_similar_hotel():
#     return get_similar_hotel

In [62]:
recommend_agent = Agent(
    name="酒店推荐助手（recommend_agent）",
    model=checkpoint,
    instructions=(
        "你的任务是根据用户的请求为用户推荐合适的酒店，包括：<推荐酒店>、<找相似酒店>。"
        "当用户的意图是为他<推荐酒店>时，调用<get_recommend_hotel>工具，并为用户总结推荐理由；"
        "当用户的意图是以酒店<找相似酒店>时，调用<get_similar_hotel>工具，并为用户总结推荐理由。"
        "注意事项：不要与用户分享你的思维过程！不要擅自替用户做出不合理的假设！"
    ),
    functions=[get_recommend_hotel, get_similar_hotel]
)

def transfer_to_recommend_agent():
    return recommend_agent

In [66]:
master_agent = Agent(
    name="意图识别助手",
    model=checkpoint,
    instructions=(
        "你的任务是对用户的请求进行意图识别，并将用户的请求转移到正确的意图工具上。"
        "用户的意图有三种：<基本对话>、<差旅知识问答>、<酒店推荐>。"
        "<基本对话>包括：<闲聊>、<询问天气>、<查询距离或路线>;"
        "<差旅知识问答>包括：<企业差旅政策与制度>、<酒店设施与政策>、<差旅业务FAQ>;"
        "<酒店推荐>包括：<推荐酒店>、<找相似酒店>。"
        "当你准备好将请求转移到对应的意图时，调用工具进行转移。"
        "当用户的请求超出意图范围时，直接以差旅秘书的身份与用户聊天。"
        "注意事项：不要与用户分享你的思维过程！不要擅自替用户做出不合理的假设！"
        ),
    functions=[transfer_to_dialog_agent, transfer_to_dialog_agent, transfer_to_recommend_agent]
)

In [49]:
system_prompt = (
    "你叫小慧助手，是由Lukas开发的差旅智能客服。"
    "你的身份是一名差旅秘书，"
    "你的任务是为用户提供基本对话、差旅知识问答、酒店推荐服务。"
    "当问及你的模型参数时，标准回答是属于公司保密信息，要强调模型设计的高效，能够提供高质量的服务。"
    "You are a helpful assistant on business travel."
)

In [68]:
departCityName = "南京"
arriveCityName = "深圳"
startDate = "2025-02-02"
endDate = "2025-02-10"
travelPurpose = "深圳基地培训"
# query = "你好"
# query = "你是什么模型？"
# query = "南京的天气"
# query = "帮我查下上海的天气"
query = "未来天气是否适合出差？"
# query = "我的住宿标准是多少？"
# query = "帮我推荐个酒店"
# query = "吴彦祖是谁？"
# query = "我今天心情不好"
# query = "和女朋友吵架了怎么办"

user_prompt = (
    f"用户咨询信息：{query}\n"
    f"出差背景信息：\n"
    f"出发城市：{departCityName}\n"
    f"到达城市：{arriveCityName}\n"
    f"停留时间：{startDate}至{endDate}\n"
    f"出差目的：{travelPurpose}\n"
    
)

In [69]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [87]:
response = swarm_client.run(
    agent=master_agent,
    messages=messages
)

In [88]:
print(response.messages[-1]["content"])

今天深圳的天气是多云，白天最高温度为24℃，晚上最低气温降至16℃。未来几天都会保持这样的舒适天气，都是多云到少云，并且风力较小，请适时调整自己的着装哦！如果有计划出行的话，记得带上太阳镜以防紫外线。
